### Commands

In [2]:
from firecloud import api as firecloud_api
import fc_interface

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

In [336]:
### Prepare cohorts for export to FC
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [358]:
# # Delete cohorts
# cohort_codes = pd.read_table('cohort_files/cohort_names_dictionary.txt', header=None)
# for coh_code in cohort_codes.loc[:, 1].tolist():
#     res = fc_interface.delete_sample_set(namespace, workspace, coh_code)

In [338]:
# ### Upload metadata for given batch
# fc_interface.update_batch_metadata('TSCA21', \
#                                   '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
#                                       paths_to_samples_info, \
#                                         namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [361]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [359]:
# Cohorts
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

---

---

In [5]:
batches_info

,tsca_id,path_to_samples_info
0,TSCA1213,/xchip/clf/seq_data/processed_for_fh/tsca1213_...
1,TSCA14,/xchip/clf/seq_data/processed_for_fh/tsca14_20...
2,TSCA15,/xchip/clf/seq_data/processed_for_fh/tsca15_20...
3,TSCA16,/xchip/clf/seq_data/processed_for_fh/tsca16_20...
4,TSCA17,/xchip/clf/seq_data/processed_for_fh/tsca17_20...
5,TSCA18,/xchip/clf/seq_data/processed_for_fh/tsca18_20...
6,TSCA19,/xchip/clf/seq_data/processed_for_fh/tsca19_20...
7,TSCA20,/xchip/clf/seq_data/processed_for_fh/tsca20_20...
8,TSCA21,/xchip/clf/seq_data/processed_for_fh/tsca21_20...


In [6]:
paths_to_samples_info = "paths_to_batches_info.xlsx"

In [25]:
batch_id='TSCA14'

In [28]:
paths_to_samples = pd.read_excel(paths_to_samples_info, index_col=0)
path_to_samples_info  = paths_to_samples.loc[batch_id, 'path_to_samples_info']
all_samples = pd.read_table(path_to_samples_info)

In [29]:
normal_samples = all_samples[all_samples['sample_type'] == "Normal"]

In [36]:
paths_to_samples.loc[~paths_to_samples.index.isin([batch_id])]

,path_to_samples_info
tsca_id,
TSCA1213,/xchip/clf/seq_data/processed_for_fh/tsca1213_...
TSCA15,/xchip/clf/seq_data/processed_for_fh/tsca15_20...
TSCA16,/xchip/clf/seq_data/processed_for_fh/tsca16_20...
TSCA17,/xchip/clf/seq_data/processed_for_fh/tsca17_20...
TSCA18,/xchip/clf/seq_data/processed_for_fh/tsca18_20...
TSCA19,/xchip/clf/seq_data/processed_for_fh/tsca19_20...
TSCA20,/xchip/clf/seq_data/processed_for_fh/tsca20_20...
TSCA21,/xchip/clf/seq_data/processed_for_fh/tsca21_20...


In [46]:
dfs = []
for i, row in paths_to_samples.loc[~paths_to_samples.index.isin([batch_id])].iterrows():
    print(row)
    df_tmp = pd.read_table(row['path_to_samples_info'])
    dfs.append(df_tmp)
df = pd.concat(dfs, axis=0)

path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca1213_...
Name: TSCA1213, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca15_20...
Name: TSCA15, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca16_20...
Name: TSCA16, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca17_20...
Name: TSCA17, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca18_20...
Name: TSCA18, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca19_20...
Name: TSCA19, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca20_20...
Name: TSCA20, dtype: object
path_to_samples_info    /xchip/clf/seq_data/processed_for_fh/tsca21_20...
Name: TSCA21, dtype: object


In [53]:
paths_to_samples = pd.read_excel(paths_to_samples_info, index_col=0)
path_to_samples_info  = paths_to_samples.loc[batch_id, 'path_to_samples_info']
all_samples = pd.read_table(path_to_samples_info)
normal_samples = all_samples[all_samples['sample_type'] == "Normal"]

# Iterate over batches except the one in question (to select N random normal samples from them)
dfs = []
for i, row in paths_to_samples.loc[~paths_to_samples.index.isin([batch_id])].iterrows():
    df_tmp = pd.read_table(row['path_to_samples_info'])
    dfs.append(df_tmp)
df = pd.concat(dfs, axis=0)

other_normal_samples = df[df['sample_type'] == "Normal"].sample(20)

In [55]:
normal_samples.columns

Index(['sample_id', 'individual_id', 'clean_bam_file_capture',
       'external_id_validation', 'aggregation_product_name_validation',
       'bsp_sample_id_validation', 'stock_sample_id_validation', 'sample_type',
       'picard_aggregation_type_validation', 'processed_subtype_validation',
       'source_subtype_validation', 'squid_sample_id_validation',
       'tumor_subtype', 'short_letter_code'],
      dtype='object')

In [56]:
other_normal_samples.columns

Index(['sample_id', 'individual_id', 'clean_bam_file_capture',
       'external_id_validation', 'aggregation_product_name_validation',
       'bsp_sample_id_validation', 'stock_sample_id_validation', 'sample_type',
       'picard_aggregation_type_validation', 'processed_subtype_validation',
       'source_subtype_validation', 'squid_sample_id_validation',
       'tumor_subtype', 'short_letter_code'],
      dtype='object')

In [60]:
name = 'PoN_TSCA14_plus_20_random'
pon = pd.concat([normal_samples, other_normal_samples], axis=0)
pon['membership:sample_set_id'] = name
pon = pon[[]]

,sample_id,individual_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code
2,AB056-Normal-SM-DB2JM,AB056,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,AB056N_primary,TSCA Rapid Cancer Detection Panel v2,SM-DB9KT,SM-DB2JM,Normal,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,AB056N_primary,Primary,NDNA
33,EH016-Normal-SM-DB2JP,EH016,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,EH016N_RETM_15FBS_p5,TSCA Rapid Cancer Detection Panel v2,SM-DB9KW,SM-DB2JP,Normal,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",EH016N_RETM_15FBS_p5,NaN,NCELL
36,EH019-Normal-SM-DB2J3,EH019,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,EH019N_primary,TSCA Rapid Cancer Detection Panel v2,SM-DB9KA,SM-DB2J3,Normal,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,EH019N_primary,NaN,NDNA
38,EH020-Normal-SM-DB2IQ,EH020,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,EH020N_primary,TSCA Rapid Cancer Detection Panel v2,SM-DB9JX,SM-DB2IQ,Normal,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,EH020N_primary,NaN,NDNA
45,JL58-Normal-SM-DB2JJ,JL58,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,JL58N_RETM_p5,TSCA Rapid Cancer Detection Panel v2,SM-DB9KQ,SM-DB2JJ,Normal,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",JL58N_RETM_p5,NaN,NCELL
48,JL59-Normal-SM-DB2J6,JL59,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,JL59N_primary,TSCA Rapid Cancer Detection Panel v2,SM-DB9KD,SM-DB2J6,Normal,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,JL59N_primary,NaN,NDNA
50,JL61-Normal-SM-DB2JI,JL61,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,JL61N_primary,TSCA Rapid Cancer Detection Panel v2,SM-DB9KP,SM-DB2JI,Normal,PCR,Tissue:Tissue Lysate/Homogenate,Tissue:Fresh Tissue,JL61N_primary,NaN,NDNA
57,PEDS153-Normal-SM-DB2IU,PEDS153,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,PEDS153N_2_RETM_p10,TSCA Rapid Cancer Detection Panel v2,SM-DB9K2,SM-DB2IU,Normal,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",PEDS153N_2_RETM_p10,NaN,NCELL
59,PEDS157-Normal-SM-DB2I6,PEDS157,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,PEDS157N_AR5_p5,TSCA Rapid Cancer Detection Panel v2,SM-DB9JD,SM-DB2I6,Normal,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",PEDS157N_AR5_p5,NaN,NCELL
62,PEDS160-Normal-SM-DB2HU,PEDS160,/xchip/clf/seq_data/processed_for_fh/tsca14_20...,PEDS160N_CM_p10,TSCA Rapid Cancer Detection Panel v2,SM-DB9J2,SM-DB2HU,Normal,PCR,Cells:Pellet frozen,"Cells:Cell Line, Viable",PEDS160N_CM_p10,NaN,NCELL
